In [1]:
import os, sys
sys.path.append('.')

from dataclasses import dataclass
from pathlib import Path

import draccus
import torch
import torch.distributed as dist
import tqdm
import wandb
from accelerate import PartialState
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from transformers.modeling_outputs import CausalLMOutputWithPast

from finetune.finetune_config import FinetuneConfig
from vla.base_prompter import PurePromptBuilder
from vla.utils import PaddedCollatorForActionPrediction
from vla.action_tokenizer import ActionTokenizer, RLbenchActionTokenizer
from vla.dataset import save_dataset_statistics, RLbenchDataset
import numpy as np
import torch.nn.functional as F

2024-07-16 19:34:07.252328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 19:34:07.252371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 19:34:07.254503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 19:34:07.262525: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 19:34:08.317075: W tensorflow/compiler/tf2

In [2]:
cfg = FinetuneConfig()

In [3]:
print(f"Fine-tuning OpenVLA Model `{cfg.vla_path}` on `{cfg.dataset_name}`")

# [Validate] Ensure GPU Available & Set Device / Distributed Context
assert torch.cuda.is_available(), "Fine-tuning assumes at least one GPU is available!"
distributed_state = PartialState()
torch.cuda.set_device(device_id := distributed_state.local_process_index)
torch.cuda.empty_cache()

# Configure Unique Experiment ID & Log Directory
exp_id = (
    f"{cfg.vla_path.split('/')[-1]}+{cfg.dataset_name}"
    f"+b{cfg.batch_size * cfg.grad_accumulation_steps}"
    f"+lr-{cfg.learning_rate}"
)
if cfg.use_lora:
    exp_id += f"+lora-r{cfg.lora_rank}+dropout-{cfg.lora_dropout}"
if cfg.use_quantization:
    exp_id += "+q-4bit"

# Start =>> Build Directories
run_dir, adapter_dir = cfg.run_root_dir / exp_id, cfg.adapter_dir / exp_id
os.makedirs(run_dir, exist_ok=True)

# Quantization Config =>> only if LoRA fine-tuning
quantization_config = None
if cfg.use_quantization:
    assert cfg.use_lora, "Quantized training only supported for LoRA fine-tuning!"
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type="nf4", #llm_int8_skip_modules = ['projector'],
    )

# Load OpenVLA Processor and Model using HF AutoClasses
processor = AutoProcessor.from_pretrained(cfg.vla_path, trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    cfg.vla_path_q,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa",
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map = "auto"
)


Fine-tuning OpenVLA Model `/media/lawrence/Work/checkpoints/openvla-7b` on `pick_up_cup`


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/lawrence/anaconda3/lib/python3.11/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [4]:
# Device Placement =>> note that BitsAndBytes automatically handles for quantized training
if cfg.use_quantization:
    vla = prepare_model_for_kbit_training(vla)
else:
    vla = vla.to(device_id)

# [LoRA] Wrap Model w/ PEFT `LoraConfig` =>> by default we set `target_modules=all-linear`
if cfg.use_lora:
    lora_config = LoraConfig(
        r=cfg.lora_rank,
        lora_alpha=min(cfg.lora_rank, 16),
        lora_dropout=cfg.lora_dropout,
        target_modules="all-linear",
        init_lora_weights="gaussian",
    )
    vla = get_peft_model(vla, lora_config)
    vla.print_trainable_parameters()

# # Wrap VLA in PyTorch DDP Wrapper for Multi-GPU Training
# vla = DDP(vla, device_ids=[device_id], find_unused_parameters=True, gradient_as_bucket_view=True)

# Create Optimizer =>> note that we default to a simple constant learning rate!
trainable_params = [param for param in vla.parameters() if param.requires_grad]
optimizer = AdamW(trainable_params, lr=cfg.learning_rate)


trainable params: 27,707,072 || all params: 7,568,944,256 || trainable%: 0.3661


In [5]:
# Create Action Tokenizer
action_tokenizer = RLbenchActionTokenizer(processor.tokenizer)

trainset = RLbenchDataset(
    cfg.train_data_path,
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
)

validset = RLbenchDataset(
    cfg.valid_data_path,
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
)

testset = RLbenchDataset(
    cfg.test_data_path,
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
)

# Create Collator and DataLoader
collator = PaddedCollatorForActionPrediction(
    processor.tokenizer.model_max_length, processor.tokenizer.pad_token_id, padding_side="right"
)

train_dataloader = DataLoader(
    trainset,
    batch_size=cfg.batch_size,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)
valid_dataloader = DataLoader(
    validset,
    batch_size=4,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)
test_dataloader = DataLoader(
    testset,
    batch_size=cfg.batch_size,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)


In [6]:
step_idx, batch = next(enumerate(train_dataloader))
batch['labels'].shape

torch.Size([2, 28])

In [7]:
# vla.train()
# optimizer.zero_grad()

# step_idx, batch = next(enumerate(train_dataloader))

# with torch.autocast("cuda", dtype=torch.bfloat16):
#     vla.train()
#     output: CausalLMOutputWithPast = vla(
#         input_ids=batch["input_ids"].to(device_id),
#         attention_mask=batch["attention_mask"].to(device_id),
#         pixel_values=batch["pixel_values"].to(torch.bfloat16).to(device_id),
#         labels=batch["labels"],
#     )
#     train_nll_loss = output.loss



In [8]:
# # Compute Accuracy and L1 Loss for Logging
# action_logits = output.logits[:, vla.vision_backbone.featurizer.patch_embed.num_patches:-1]
# action_gt = batch["labels"][:, 1:].to(device_id)
# mask = action_gt >= action_tokenizer.action_token_begin_idx
# masked_logits = action_logits[mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(2,7,-1)

# # Compute L1 Loss on Predicted (Continuous) Actions
# action_pred = action_tokenizer.output_logit_to_continous_action(masked_logits)
# action_gt = batch["actions"].to(device_id)
# train_l1_loss = torch.nn.functional.l1_loss(action_pred, action_gt)

In [9]:
for batch in valid_dataloader:
    with torch.autocast("cuda", dtype=torch.bfloat16):
        output: CausalLMOutputWithPast = vla(
            input_ids=batch["input_ids"].to(device_id),
            attention_mask=batch["attention_mask"].to(device_id),
            pixel_values=batch["pixel_values"].to(torch.bfloat16).to(device_id),
            labels=batch["labels"],
        )

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 

: 

In [ ]:
action_tokenizer.tokenizer.vocab_size - action_preds[mask] - 1

NameError: name 'action_preds' is not defined

In [ ]:
(352 - action_logits[0][mask[0],action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size]).shape

torch.Size([7, 352])

In [ ]:
masked_logits

tensor([[[-10.4375, -10.9375, -11.3750,  ...,  -3.3438,   0.2324,  -0.4141],
         [-11.6250,  -4.8750,  -5.7188,  ...,  -6.6250,   2.8594,   1.7812],
         [ -5.2812,  -3.4375,  -6.5000,  ...,  -1.0312,  -5.0312,  -0.1660],
         ...,
         [ -7.1562,  -3.6562,  -5.4688,  ...,   6.8125,  -4.9375,   6.3125],
         [ -5.1250,  -5.1250,  -2.7031,  ...,   6.0625,  -3.2188,  -1.8516],
         [ -8.3125,  -2.8906,  -5.3438,  ...,   2.4531,   6.1250,   1.8984]],

        [[-10.0625,  -7.7812,  -9.5625,  ...,  -7.5312,  -3.1406,  -0.7734],
         [ -7.3438,  -5.2188,  -6.2500,  ...,  -1.4453,  -4.1250,  -2.0000],
         [ -6.2188,  -6.9375,  -8.1875,  ...,   0.4023,   1.3594,   7.9688],
         ...,
         [ -5.5000,  -7.4062,  -6.0312,  ...,  -4.2188,   0.3633,  11.8750],
         [ -4.0625,  -4.7812,  -4.3438,  ...,   8.5000,   2.0938,  -0.3750],
         [ -6.9062,  -4.6562,  -6.5000,  ...,   5.1875,   1.2969,   2.6562]]],
       device='cuda:0', grad_fn=<ViewBackwar

In [ ]:
masked_logits.shape

torch.Size([2, 7, 352])

In [ ]:
pred_action = action_tokenizer.output_logit_to_continous_action(masked_logits,batch['actions'])

In [ ]:
F.l1_loss(pred_action, batch['actions'].to(pred_action.device))

tensor(0.7198, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
masked_logits[:,1,:50].dtype

torch.float32